In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import datetime
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
from scipy.optimize import curve_fit
import glob
import os
import shutil
import plotly.graph_objects as go
from openpyxl import load_workbook

### find and read the data
replace the file path with the one you want to analyze

In [3]:
nox_path='data/T200_NOx_/2023-09-26_08.33.51_PC14751_T200_NOx.dat'
no2_path='data/NO2_ICAD/2023_09_26_Channel1_Data.dat'
hono_path='data/HONO_ICAD/2023_09_26_Channel1_Data.dat'
rh_path='data/Humidity_Sensors_/2023-09-26_08.33.52_PC14751_Humidity_Sensors.dat'

In [4]:
hono=pd.read_csv(hono_path,sep = '\t', engine='python') #HONO from ICAD
no2=pd.read_csv(no2_path,sep = '\t', engine='python') #NO2 from ICAD
nox=pd.read_csv(nox_path,sep = '\t', header=1, engine='python') #no of nox analyzer
rh=pd.read_csv(rh_path,sep = '\t', header=1, encoding= 'unicode_escape', engine='python') #RH data

#ICAD
hono['Start Date/Time (UTC)']=pd.to_datetime(hono['Start Date/Time (UTC)'])
no2['Start Date/Time (UTC)']=pd.to_datetime(no2['Start Date/Time (UTC)'])

#NOx analyzer
nox['datetime']=nox['Date']+ nox['Time']
nox['time']= pd.to_datetime(nox['datetime'], format='%d.%m.%Y%H:%M:%S') 


#RH sensor
rh['datetime']=rh['Date']+ rh['Time']
rh['time']= pd.to_datetime(rh['datetime'], format='%d.%m.%Y%H:%M:%S') 

hono['Time_correct']=hono['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
hono['Time_correct']= pd.to_datetime(hono['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

no2['Time_correct']=no2['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
no2['Time_correct']= pd.to_datetime(no2['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

print(hono['Time_correct'][1])

2023-09-26 02:01:18.774000


# calculate dilution factor

In [5]:
# dil_flow=2.2435
# flow_tube=1.7224
# tot_flow=flow_tube+dil_flow
# dil_factor=tot_flow/flow_tube
dil_factor=1

## select data useful in new dataframes

In [6]:

df_hono_icad=pd.DataFrame()
df_no2_icad=pd.DataFrame()
df_rh=pd.DataFrame()
df_nox_t200=pd.DataFrame()

df_hono_icad['time']=hono['Time_correct']
df_no2_icad['time']=no2['Time_correct']
df_nox_t200['time']=nox['time']
df_rh['time']=rh['time']
df_rh['dew_point']=rh['DP7[°C]']
df_hono_icad['hono']=hono['HONO (ppb)']*dil_factor
df_no2_icad['no2']=no2['NO2 (ppb)']*dil_factor
df_nox_t200['no']=nox['NO']*dil_factor


## get average time at a certain time interval

In [7]:
average_interval='60s'
df_hono_icad=df_hono_icad.resample(average_interval,on='time').mean()
df_hono_icad['date_time']=df_hono_icad.index

df_no2_icad=df_no2_icad.resample(average_interval,on='time').mean()
df_no2_icad['date_time']=df_no2_icad.index
df_no2_icad['date_time']= pd.to_datetime(df_no2_icad['date_time']) 

df_rh=df_rh.resample(average_interval,on='time').mean()
df_rh['date_time']=df_rh.index

df_nox_t200=df_nox_t200.resample(average_interval,on='time').mean()
df_nox_t200['date_time']=df_nox_t200.index


# combine all data  in one dataframe; align with time and plot them together

In [158]:
df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-09-26 09:08:00'])

# df.hono=df.hono-df.hono[46:76].mean()
time_start=35-10
# df=df[time_start:time_start+10+41+10]
df=df[time_start:time_start+60]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
# x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x= x,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=df.no-0.175,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))

# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title='0926a NaNO3 only 0.1M 600 uL',titlefont=dict(color="black",size=25))
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', 
               range=[-2,68],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.9 ,xanchor="left",x=0.75,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)


fig_1.add_vrect(x0=9, x1=40,annotation_text="UV-A lights on",annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}

fig_1.show(config=config)
# fig_1.to_image('3.jpeg')


        hono           date_time       no2   no  dew_point
35  0.073333 2023-09-26 09:08:00 -0.006833  0.8  16.803448


          Do we need to calibrate T2OO NOx analyzer; it looks weird...
          it is currently not connected to a Na2co3 before sampling

In [9]:

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')

print(df[df['date_time'] == '2023-09-26 10:21:00'])


         hono           date_time       no2    no  dew_point
108 -0.132333 2023-09-26 10:21:00  0.059767  0.45  17.631034


In [10]:

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-09-26 09:21:00'])

# df.hono=df.hono-df.hono[46:76].mean()
time_start=108-10
# df=df[time_start:time_start+10+41+10]
df=df[time_start:time_start+110]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x= df.date_time,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title='0926b NaNO3 (0.1M) + DHB (0.02M) 600 uL',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', 
              #  range=[-2,100],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.9 ,xanchor="left",x=0.75,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

# fig_1.add_vrect(
#     x0=11, x1=11+32,
#     fillcolor="yellow", opacity=0.3,
#     layer="below", line_width=0,
# )

config = {'displayModeBar': True}

fig_1.show(config=config)
# fig_1.to_image('3.jpeg')


        hono           date_time       no2    no  dew_point
48  1.863333 2023-09-26 09:21:00  0.715817  0.38       16.5


		## here shows, first adding O2 improves nitrate photolysis, but adding more O2 decreases all the conc.; the change in the concentration is more likely due to the change of total purging flow.
		
		exp 0926B	solution 0925a	nitrate+DHB

		26/09/2023 10:21:53	let the sample dry in pure N2	

		26/09/2023 10:36:56	LIGHTS ON 	NO O2

		26/09/2023 10:59:26	switch to add O2

		adjust N2 and O2 ratio	 
		
		## pay attention: although I tried to keep the total cwft flow to be 2300 ml min-1, the flow still can be different when I change the N2/O2 ratio. 

		26/09/2023 11:26:18	N2/O2 = 1:1	N2 = 1150 ml min-1
				O2=1150 ml min-1 

		26/09/2023 11:36:11	only N2	

		26/09/2023 11:42:07	lights off	

		note: the solution is one day old
		

In [11]:

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-09-26 12:09:00'])

         hono           date_time       no2     no  dew_point
216  0.115167 2023-09-26 12:09:00  0.056033  1.615  17.310345


In [12]:

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-09-26 09:21:00'])

# df.hono=df.hono-df.hono[46:76].mean()
time_start=216-10
# df=df[time_start:time_start+10+41+10]
df=df[time_start:time_start+90]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x= df.date_time,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title='0926c NaNO3 (0.1M) + DHB (0.02M) 600 uL',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', 
              #  range=[-2,100],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.9 ,xanchor="left",x=0.75,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

# fig_1.add_vrect(
#     x0=11, x1=11+32,
#     fillcolor="yellow", opacity=0.3,
#     layer="below", line_width=0,
# )

config = {'displayModeBar': True}

fig_1.show(config=config)
# fig_1.to_image('3.jpeg')


        hono           date_time       no2    no  dew_point
48  1.863333 2023-09-26 09:21:00  0.715817  0.38       16.5


          exp 0926c	solution 0925a	nitrate+DHB

          26/09/2023 12:00:26	

          26/09/2023 12:09:50	LIGHTS ON 	1150*2 ml min-1 N2

          26/09/2023 12:40:55	add half O2	N2 = 1150 ml min-1

                    O2=1150 ml min-1 

          26/09/2023 13:10:42	ONLY N2	
          
                            
          26/09/2023 14:21:01	cwft in line	N2 = 1150 ml min-1

          26/09/2023 14:25:32	lights on 	O2=1150 ml min-1 

          26/09/2023 14:37:16	lights off	

          26/09/2023 14:40:47	bypass chamber	

In [29]:

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-09-26 15:14:00'])

# df.hono=df.hono-df.hono[46:76].mean()
time_start=401-10
# df=df[time_start:time_start+10+41+10]
df=df[time_start:time_start+90]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x= df.date_time,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title='0926d empty tube 1st (washed, but still not clean)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', 
              #  range=[-2,100],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.9 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

# fig_1.add_vrect(
#     x0=11, x1=11+32,
#     fillcolor="yellow", opacity=0.3,
#     layer="below", line_width=0,
# )

config = {'displayModeBar': True}

fig_1.show(config=config)
# fig_1.to_image('3.jpeg')


         hono           date_time      no2     no  dew_point
401  1.155167 2023-09-26 15:14:00  0.20898  2.765  15.668966


In [30]:

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-09-26 16:59:00'])

# df.hono=df.hono-df.hono[46:76].mean()
time_start=485-10
# df=df[time_start:time_start+10+41+10]
df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x= df.date_time,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title='0926d empty tube 2nd (washed completely)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', 
              #  range=[-2,100],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.9 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

# fig_1.add_vrect(
#     x0=11, x1=11+32,
#     fillcolor="yellow", opacity=0.3,
#     layer="below", line_width=0,
# )

config = {'displayModeBar': True}

fig_1.show(config=config)
# fig_1.to_image('3.jpeg')


          hono           date_time     no2    no  dew_point
506  10.124167 2023-09-26 16:59:00  33.925  3.23  15.489655


In [138]:

# df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-09-26 18:04:00'])

# df.hono=df.hono-df.hono[46:76].mean()
time_start=910-10
# df=df[time_start:time_start+110]
df=df[time_start:955]


x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes
x= df.x_minutes
# x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono-1.5,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=46.4-df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="△NO<sub>2</sub>"))

# fig_1.add_trace(go.Scatter(x= df_nox_t200.x_minutes,y=df_nox_t200.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title='0926f DHB only (0.02M) 600 uL',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', 
               range=[-3,55],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.6 ,xanchor="left",x=0.85,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig_1.add_vrect(x0=5, x1=5+6,annotation_text="cwft in line",annotation_position="top",annotation_font_size=14,fillcolor="grey", opacity=0.3,layer="below", line_width=0)
fig_1.add_vrect(x0=11, x1=36,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)
fig_1.add_vrect(x0=36, x1=48,annotation_text="UV-A lights off", annotation_position="top",annotation_font_size=14,fillcolor="grey", opacity=0.3,layer="below", line_width=0)
fig_1.add_vrect(x0=49 , x1=48+5,annotation_text="bypass", annotation_position="top",annotation_font_size=14,fillcolor="white", opacity=0.3,layer="below", line_width=0)


config = {'displayModeBar': True}

fig_1.show(config=config)
# fig_1.to_image('3.jpeg')

         hono           date_time     no2
963  3.001833 2023-09-26 18:04:00  46.411


In [139]:

fig_1= go.Figure()
# fig_1.add_trace(go.Scatter(x=x,y=(df.hono-1.5)/(46.4-df.no2)*100,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x=x,y=(46.4-df.no2)/46.4*100,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))

# fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
# fig_1.add_trace(go.Scatter(x= x,y=,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="△NO<sub>2</sub>"))

# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title='0926g empty tube 3rd (washed completely)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', 
               # range=[-2,55],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig_1.add_vrect(x0=9, x1=14,annotation_text="cwft in line",annotation_position="top",annotation_font_size=14,fillcolor="grey", opacity=0.3,layer="below", line_width=0)
fig_1.add_vrect(x0=14, x1=14+16,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)
fig_1.add_vrect(x0=30, x1=36,annotation_text="UV-A lights off", annotation_position="top",annotation_font_size=14,fillcolor="grey", opacity=0.3,layer="below", line_width=0)
fig_1.add_vrect(x0=36 , x1=36+10,annotation_text="bypass", annotation_position="top",annotation_font_size=14,fillcolor="white", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
# fig_1.to_image('3.jpeg')


In [141]:
46.4*0.21-46.4

-36.656

In [142]:
# df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-09-26 17:59:00'])

# df.hono=df.hono-df.hono[46:76].mean()
time_start=959-10
df=df[time_start:time_start+45]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
# x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono-1.5,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=48.3-df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="△NO<sub>2</sub>"))

# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title='0926g empty tube 3rd (washed completely)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', 
               range=[-2,55],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig_1.add_vrect(x0=9, x1=14,annotation_text="cwft in line",annotation_position="top",annotation_font_size=14,fillcolor="grey", opacity=0.3,layer="below", line_width=0)
fig_1.add_vrect(x0=14, x1=14+16,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)
fig_1.add_vrect(x0=30, x1=36,annotation_text="UV-A lights off", annotation_position="top",annotation_font_size=14,fillcolor="grey", opacity=0.3,layer="below", line_width=0)
fig_1.add_vrect(x0=36 , x1=36+10,annotation_text="bypass", annotation_position="top",annotation_font_size=14,fillcolor="white", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
# fig_1.to_image('3.jpeg')


       hono           date_time        no2
958  1.4905 2023-09-26 17:59:00  48.167333


In [146]:

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=(df.hono-1.5)/(48.3-df.no2)*100,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
# fig_1.add_trace(go.Scatter(x=x,y=(48.3-df.no2)/48.3*100,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))

# fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
# fig_1.add_trace(go.Scatter(x= x,y=,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="△NO<sub>2</sub>"))

# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title='0926g empty tube 3rd (washed completely)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', 
               # range=[-2,55],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig_1.add_vrect(x0=9, x1=14,annotation_text="cwft in line",annotation_position="top",annotation_font_size=14,fillcolor="grey", opacity=0.3,layer="below", line_width=0)
fig_1.add_vrect(x0=14, x1=14+16,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)
fig_1.add_vrect(x0=30, x1=36,annotation_text="UV-A lights off", annotation_position="top",annotation_font_size=14,fillcolor="grey", opacity=0.3,layer="below", line_width=0)
fig_1.add_vrect(x0=36 , x1=36+10,annotation_text="bypass", annotation_position="top",annotation_font_size=14,fillcolor="white", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
# fig_1.to_image('3.jpeg')
